# Pour le polluant NO2

### On importe les bibliothèques

In [3]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Conversion des données en DataFrame

In [6]:

# URL de base
url = "https://data.airpl.org/api/v1/mesure/horaire/"

# Paramètres de base de la requête
base_params = {
    "code_configuration_de_mesure__code_point_de_prelevement__code_polluant": "03",
    "date_heure_tu__range": "2024-1-1,2024-3-31 23:00:00",
    "export": "json"
}
# Limite de résultats par requête
limit = 1000
offset = 0

# DataFrame pour stocker tous les résultats
dfNO2 = pd.DataFrame()

while True:
    # Mettre à jour les paramètres avec la limite et l'offset
    params = base_params.copy()
    params.update({
        "limit": limit,
        "offset": offset
    })
    
    # Récupérer les données
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        df_results = pd.DataFrame(data['results'])  # Adapter selon la structure des données JSON
        
        # Ajouter les résultats au DataFrame principal
        dfNO2 = pd.concat([dfNO2, df_results], ignore_index=True)
        
        # Vérifier si le nombre de résultats récupérés est inférieur à la limite
        if len(df_results) < limit:
            break  # Arrêter la boucle si tous les enregistrements ont été récupérés
        
        # Mettre à jour l'offset pour la prochaine itération
        offset += limit
    else:
        print(f"Erreur {response.status_code}: {response.text}")
        break

print(f"Total records retrieved: {len(dfNO2)}")

# Sauvegarder le DataFrame pour utilisation ultérieure
dfNO2.to_pickle('PMNO2.pkl')

# Afficher les premières lignes du DataFrame
print(dfNO2.head())


In [7]:
# Charger le DataFrame depuis le fichier sauvegardé
dfNO2 = pd.read_pickle('NO2.pkl')

# Afficher les types de chaque colonne
print(dfNO2.dtypes)

In [8]:
print(dfNO2.head(10))

## Analyse et correction du dataset

### Types et valeurs manquantes par variable

In [9]:
dfNO2.shape

In [10]:
dfNO2.info

In [11]:
dfNO2.isnull().sum()

Observations : 
- On remarque que pour les colonnes "valeur_originale" et "valeur" on a quelques lignes vides (23), mais aucune colonne entièrement vide

## Visualiser les différentes données de chaque colonne

### Pour les colonnes de type float64

In [12]:
for col in dfNO2.select_dtypes('float64'):
    print(f'{col :-<20} {dfNO2[col].unique()}')

In [13]:
for col in dfNO2.select_dtypes('int64'):
    print(f'{col :-<20} {dfNO2[col].unique()}')

In [14]:
for col in dfNO2.select_dtypes('object'):
    print(f'{col :-<20} {dfNO2[col].unique()}')

Observations : 
- on remarque que pour les colonnes de type float64, pour les colonnes "valeur_originale" et "valeur", des valeurs ne sont pas correctes car il y à une présence de "nan" et "-999.".
- Nous allons donc supprimer les lignes ou ces dernières sont présentes, et donc supprimer toutes les valeurs qui seront négatives pour englober.

Certaines colonnes ne semblent pas être du bon types, il faut donc les changer : 
- code_polluant : object --> int64
- code_commune : object --> int64
- departement_code : object --> int64

### Supression des valeurs "nan" et "-999" pour les colonnes "valeur_originale" et "valeur"

In [15]:
# Assurez-vous que dfNO2 est bien chargé et nettoyé

# Supprimer les lignes comportant des valeurs "NaN" dans les colonnes "valeur" et "valeur_originale"
dfNO2.dropna(subset=['valeur', 'valeur_originale'], inplace=True)

# Supprimer les lignes comportant des valeurs négatives dans les colonnes "valeur" et "valeur_originale"
dfNO2 = dfNO2[(dfNO2['valeur'] >= 0) & (dfNO2['valeur_originale'] >= 0)]

# Afficher les premières lignes du DataFrame pour vérifier le nettoyage
print(dfNO2.head())


In [16]:
for col in dfNO2.select_dtypes('float64'):
    print(f'{col :-<20} {dfNO2[col].unique()}')

### Changement de type pour les colonnes : 
- code_polluant : object --> int64
- code_commune : object --> int64
- departement_code : object --> int64

In [17]:
# Vérifier si les colonnes existent dans le DataFrame
colonnes_a_convertir = ['code_commune', 'departement_code', 'code_polluant']

# Convertir les colonnes en int64
for col in colonnes_a_convertir:
    if col in dfNO2.columns:
        dfNO2[col] = dfNO2[col].astype('int64')

# Afficher les types de colonnes pour vérifier les conversions
print(dfNO2.dtypes)

In [18]:
from unidecode import unidecode
# Convertir les valeurs de la colonne 'nom_commune' en minuscules et supprimer les accents
dfNO2['nom_commune'] = dfNO2['nom_commune'].apply(lambda x: unidecode(x).lower())